In [1]:
import pygmt 
import xarray as xr
import numpy as np
import pandas as pd
import os

In [2]:
# to monitor memory usage
from dask.distributed import Client
client = Client()
client

/Users/kang/miniconda3/envs/pygmt/lib/python3.9/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60244 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60244/status,
Dashboard: http://127.0.0.1:60244/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60245,Workers: 4
Dashboard: http://127.0.0.1:60244/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:60266,Total threads: 2
Dashboard: http://127.0.0.1:60267/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:60251,


### 1.0 Define function to preprocess the chla

In [5]:
def preprocess_chla(x):
    
    # to rearrange the latitude and longitude to have equal spacing 
    latitude = np.arange(-90+((180/4320)/2),90+((180/4320)/2),180/4320)
    latitude = latitude[::-1]
    longitude = np.arange(-180+((360/8640)/2),180+((360/8640)/2),360/8640)
    
    chl=x.CHL[0].data
    
    time=x.time
    chl = np.expand_dims(chl,axis=0)

    
    da = xr.DataArray(data=chl,dims=["time","lat","lon"],coords=dict(lat=(['lat'],latitude),lon=(['lon'],longitude),time=time)).rename('CHL')
    da_region = da.sel(lat=slice(10,-10),lon=slice(92,122))
    return da_region

### 2.0 Load the data

In [6]:
## Load the data
### Load the data from 2002 to 2022
list_data_chla = []
for i in range (2000,2023):
    year =str(i)
    file_path_chla = f'/Volumes/WDdata/data/cmems/chla/globcolourL4/4km/month/{year}/*_P1M.nc'
    data_chla = xr.open_mfdataset(file_path_chla,preprocess=preprocess_chla,parallel=True,combine='nested',concat_dim='time')
    list_data_chla.append(data_chla)
    print('Chl-a for',year, 'loaded')
chla = xr.combine_nested(list_data_chla,concat_dim='time')

Chl-a for 2000 loaded
Chl-a for 2001 loaded
Chl-a for 2002 loaded
Chl-a for 2003 loaded
Chl-a for 2004 loaded
Chl-a for 2005 loaded
Chl-a for 2006 loaded
Chl-a for 2007 loaded
Chl-a for 2008 loaded
Chl-a for 2009 loaded
Chl-a for 2010 loaded
Chl-a for 2011 loaded
Chl-a for 2012 loaded
Chl-a for 2013 loaded
Chl-a for 2014 loaded
Chl-a for 2015 loaded
Chl-a for 2016 loaded
Chl-a for 2017 loaded
Chl-a for 2018 loaded
Chl-a for 2019 loaded
Chl-a for 2020 loaded
Chl-a for 2021 loaded
Chl-a for 2022 loaded


### 3.0 Calculate the data


In [7]:
# get the monthly climatology for past 22 years

chla_month = (chla.groupby('time.month').mean()).compute()

In [8]:
# repeat the montly climatology, for 22 years for calculation later

chla_month_22years = np.tile(chla_month,(23,1,1))

In [9]:
# repeat the montly climatology, for 22 years for calculation later

chla_month_22years = np.tile(chla_month,(23,1,1))

In [10]:
# check the len of monthly observation and monthly climatology
print('length of chla_month_22years =', len(chla_month_22years), 'and length of chla =', len(chla))

length of chla_month_22years = 276 and length of chla = 276


In [11]:
# get the anomaly for each month according to the climatology
chla_anomaly = (chla - chla_month_22years).compute()

In [12]:
# get the anomaly for each month in percentage
chla_anomaly_percent = (chla_anomaly/chla_month_22years)*100

### 4.0 Plot the data

In [13]:
n = 0
for i in range (len(chla_anomaly_percent)):

    grid = chla_anomaly_percent[i]
    n += 1
    fig = pygmt.Figure()
    pygmt.config(MAP_FRAME_TYPE="plain",MAP_TITLE_OFFSET=-0.3)
    pygmt.config(MAP_TICK_LENGTH_PRIMARY='0.25c')
    pygmt.config(FONT_ANNOT_PRIMARY="10p")
    pygmt.config(FONT_TITLE="20p,Helvetica")
    pygmt.config(FONT_LABEL=20)
    pygmt.config(MAP_FRAME_PEN="1.0p")
    pygmt.config(FONT_LABEL="10p")

    cpt =pygmt.makecpt(cmap='vik', series=[-200,200,10],background=True)
    fig.grdimage(grid=grid, projection = 'M15c',frame=["af",'+tChlorophyll-a'],cmap=cpt,region=[92,122,-5,10])
    fig.colorbar(frame=["af", "x+lChl-a Anomaly (%)"],position="JBC+o0c/0.8c+w12c/0.3c+h+e") 
    fig.coast(shorelines=True)
    fig.text(text= np.datetime_as_string(grid.time,unit="D"),x=95,y=9,font='12p',fill='white')
    
    with fig.inset(position="jTR+w2.0c+o0.2c"):
        # Create a figure in the inset using coast. This example uses the azimuthal
        # orthogonal projection centered at 47E, 20S. The land color is set to
        # "gray" and Madagascar is highlighted in "red3".
        pygmt.config(MAP_FRAME_PEN='0.01p')
        fig.coast(
            region=[92,122,-5,10],
            projection="G105/0/2.0c",
            land="gray",
            water="white",
            frame='afg')

        rectangle = [92,-5,122,10]
        fig.plot(data=[rectangle], style = 'r+s',pen="0.01p,red",projection="G105/0/2.0c")
    
    
    file_path = '/Users/kang/Documents/Portfolio/Geospatial_visualisation/Chla_anomaly/'
    series = f"{n:03}"
    number = str(series)
    fig.savefig(os.path.join(file_path+series+'.png'))
    print("Graph for " + np.datetime_as_string(grid.time,unit="D") + " Done")

Graph for 2000-01-01 Done
Graph for 2000-02-01 Done
Graph for 2000-03-01 Done
Graph for 2000-04-01 Done
Graph for 2000-05-01 Done
Graph for 2000-06-01 Done
Graph for 2000-07-01 Done
Graph for 2000-08-01 Done
Graph for 2000-09-01 Done
Graph for 2000-10-01 Done
Graph for 2000-11-01 Done
Graph for 2000-12-01 Done
Graph for 2001-01-01 Done
Graph for 2001-02-01 Done
Graph for 2001-03-01 Done
Graph for 2001-04-01 Done
Graph for 2001-05-01 Done
Graph for 2001-06-01 Done
Graph for 2001-07-01 Done
Graph for 2001-08-01 Done
Graph for 2001-09-01 Done
Graph for 2001-10-01 Done
Graph for 2001-11-01 Done
Graph for 2001-12-01 Done
Graph for 2002-01-01 Done
Graph for 2002-02-01 Done
Graph for 2002-03-01 Done
Graph for 2002-04-01 Done
Graph for 2002-05-01 Done
Graph for 2002-06-01 Done
Graph for 2002-07-01 Done
Graph for 2002-08-01 Done
Graph for 2002-09-01 Done
Graph for 2002-10-01 Done
Graph for 2002-11-01 Done
Graph for 2002-12-01 Done
Graph for 2003-01-01 Done
Graph for 2003-02-01 Done
Graph for 20

In [14]:
### 5.0 Make it into video

In [15]:
%%bash
ffmpeg -loglevel warning -f image2 -framerate 10 -y -i "/Users/kang/Documents/Portfolio/Geospatial_visualisation/Chla_anomaly/%3d.png" -vcodec libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -pix_fmt yuv420p chla_anomaly.mp4